In [47]:
#Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
from scipy.stats import sem

In [34]:
#import data
md_df = pd.read_csv('Resources/Pymaceuticals_data_Mouse_metadata.csv')
sr_df = pd.read_csv('Resources/Pymaceuticals_data_Study_results.csv')
md_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16


In [37]:
sr_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [40]:
#check the data for any mouse ID with duplicate time points and remove any data associated with that mouse ID.
sr_df = sr_df.drop_duplicates()
df = pd.merge(md_df, sr_df, on='Mouse ID')
df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [48]:
#Generate a summary statistics table consisting of the mean, median, variance, standard deviation, 
#and SEM of the tumor volume for each drug regimen.
drug_df = df.groupby(['Drug Regimen'])
tumorMean = drug_df['Tumor Volume (mm3)'].mean()
tumorMedian = drug_df['Tumor Volume (mm3)'].median()
tumorVar = drug_df['Tumor Volume (mm3)'].var()
tumorSD = drug_df['Tumor Volume (mm3)'].std()
tumorSEM = sem(drug_df['Tumor Volume (mm3)'])

tumorSEM


/Users/stevenzych/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/scipy/stats/stats.py:269: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [11]:
#Generate a bar plot using both Pandas's `DataFrame.plot()` and Matplotlib's `pyplot` 
#that shows the total number of measurements taken for each treatment regimen throughout the course of the study.

In [12]:
#Generate a pie plot using both Pandas's `DataFrame.plot()` and Matplotlib's `pyplot` 
#that shows the distribution of female or male mice in the study.

In [13]:
#Calculate the final tumor volume of each mouse across four of the most promising treatment regimens: 
#Capomulin, Ramicane, Infubinol, and Ceftamin. Calculate the quartiles and IQR and 
#quantitatively determine if there are any potential outliers across all four treatment regimens.

In [14]:
#Using Matplotlib, generate a box and whisker plot of the final tumor volume for all four treatment regimens and 
#highlight any potential outliers in the plot by changing their color and style.
#**Hint**: All four box plots should be within the same figure. Use this [Matplotlib documentation page]

In [15]:
#Select a mouse that was treated with Capomulin and generate a line plot of tumor volume vs. time point for that mouse.

In [16]:
#Generate a scatter plot of tumor volume versus mouse weight for the Capomulin treatment regimen.

In [17]:
#Calculate the correlation coefficient and linear regression model between mouse weight and 
#average tumor volume for the Capomulin treatment. Plot the linear regression model on top of the previous scatter plot.

In [ ]:
#Look across all previously generated figures and tables and write at least three observations or inferences 
#that can be made from the data. Include these observations at the top of notebook.